# Third Test

This time we have a more robust auto regressive implemenation, because now I understand the attention masking. Both full and causal.

In [ ]:
from huggingface_hub import login
login()

In [ ]:
# Clone

In [ ]:
%cd XHUMAN

We have several problem trying to use the transformers library, this requires a more careful study

In [ ]:
!uv pip install -e .[pi]

# Convert the image into something workable

In [ ]:
from transformers import AutoTokenizer
# Load tokenizer
print("Loading tokenizer...")
tokenizer = AutoTokenizer.from_pretrained("google/paligemma-3b-pt-224",
                                          use_fast=False)

Loading tokenizer...


In [ ]:
import os
import cv2

img_share_path = '/content'
img_name_list = ['top.png']
img_list = []

if img_share_path and os.path.exists(img_share_path):
    # Load images from path
    for img_name in img_name_list:
        img_path = os.path.join(img_share_path, img_name)
        if os.path.exists(img_path):
            img = cv2.imread(img_path)
            if img is not None:
                # Convert BGR to RGB
                img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
                img_list.append(img)

In [ ]:
high_level_prompt = 'Task: Put the grapes on the basket.\nSubtask:'

In [ ]:
batch_raw = {
    "images": img_list,
    "text": high_level_prompt
}


In [ ]:
from xhuman.policy.pi05_policy import PI05Policy

model_id = "NONHUMAN-RESEARCH/pi05_ki_training_fruits"

policy = PI05Policy.from_pretrained(model_id)
policy.model.eval()
policy.model.to(device)


In [ ]:
from xhuman.policy.pi05.processor_pi05 import make_pi05_pre_post_processors_ki

pre_post_processors, _ = make_pi05_pre_post_processors_ki(
    config=config,
    dataset_stats=dataset_stats
)

batch = pre_post_processors(batch_raw)

In [ ]:
batch

In [ ]:
max_decoding_steps = 100
temperature = 0.1
PALIGEMMA_EOS_TOKEN = 1

In [ ]:
subtask_tokenized = policy.model.sample_subtask(
    images=processed_images,
    img_masks=img_masks,
    tokens=tokens,
    masks=masks.bool(),
    max_decoding_steps=max_decoding_steps,
    eos_token_id=PALIGEMMA_EOS_TOKEN,
    temperature=temperature
)

In [ ]:
# _preprocess_images returns processed_images and img_masks
# processed_images: list of (B, C, H, W) tensors
# img_masks: list of (B,) boolean tensors

# Prepare language prompt

print(f"\nPrompt: {high_level_prompt}")

# Tokenize the prompt
# Add newline if not present
if not high_level_prompt.endswith('\n'):
    high_level_prompt += '\n'

tokenized = tokenizer(
    high_level_prompt,
    return_tensors="pt",
    padding=True,
    truncation=True,
    max_length=512
)

print(f"Tokenized: {tokenized}")

tokens = tokenized["input_ids"].to(device)  # (B, seq_len)
masks = tokenized["attention_mask"].to(device)  # (B, seq_len)

print(f"Tokens shape: {tokens.shape}")
print(f"Masks shape: {masks.shape}")


Prompt: Task: Put the grapes on the basket.
Subtask:
Tokenized: {'input_ids': tensor([[     2,   7071, 235292,  13298,    573,  50161,    611,    573,  12220,
         235265,    108,   3351,   8277, 235292,    108]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]])}
Tokens shape: torch.Size([1, 15])
Masks shape: torch.Size([1, 15])


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Visualize the processed images
# These images are normalized to [-1, 1] and might be resized

if 'processed_images' in locals():
    num_images = len(processed_images)
    plt.figure(figsize=(5 * num_images, 5))

    for i, img_tensor in enumerate(processed_images):
        # processed_images are typically (B, H, W, C) or (B, C, H, W)
        # We select the first image in the batch
        img = img_tensor[0].detach().cpu().numpy()

        # Check if channels are first (C, H, W) -> convert to (H, W, C) for plotting
        # Heuristic: if first dimension is 3, it's likely channels
        if img.ndim == 3 and img.shape[0] == 3:
             img = np.transpose(img, (1, 2, 0))

        # Un-normalize: [-1, 1] -> [0, 1]
        img = (img + 1.0) / 2.0
        img = np.clip(img, 0.0, 1.0)

        plt.subplot(1, num_images, i + 1)
        plt.imshow(img)
        plt.title(f"Image {i}\nShape: {img_tensor.shape}")
        plt.axis('off')

    plt.tight_layout()
    plt.show()
else:
    print("processed_images variable not found.")

In [ ]:
with torch.no_grad():
    # FIX: Ensure images are in (B, C, H, W) format
    # The error "expected input... to have 3 channels, but got 224" occurs because
    # the input was (B, H, W, C) but the model expects (B, C, H, W).
    fixed_processed_images = []
    for img in processed_images:
        # Check if shape is (B, H, W, C=3) instead of (B, C=3, H, W)
        if img.ndim == 4 and img.shape[-1] == 3 and img.shape[1] != 3:
            img = img.permute(0, 3, 1, 2)
        fixed_processed_images.append(img)

    output_tokens, updated_past_key_values = policy.model.sample_low_level_task(
        images=fixed_processed_images,
        img_masks=img_masks,
        tokens=tokens,
        masks=masks.bool(), # FIX: Convert attention mask to boolean
        max_decoding_steps=max_decoding_steps,
        eos_token_id=PALIGEMMA_EOS_TOKEN,
        temperature=temperature
    )

    print(f"Output tokens shape: {output_tokens.shape}")
    print(f"Output tokens:\n{output_tokens}")
    print(f"Updated past key values: {updated_past_key_values}")

    # Decode the generated tokens
    print("\n" + "="*80)
    print("High Level Prompt\n",high_level_prompt)
    print("Decoded output:")
    print("="*80)

    for batch_idx in range(output_tokens.shape[0]):
        batch_tokens = output_tokens[batch_idx]
        # Remove padding (zeros) and decode
        non_zero_tokens = batch_tokens[batch_tokens != 0]
        if len(non_zero_tokens) > 0:
            decoded_text = tokenizer.decode(non_zero_tokens, skip_special_tokens=False)
            print(f"Batch {batch_idx}: {decoded_text}")
        else:
            print(f"Batch {batch_idx}: (empty)")

print("\n" + "="*80)
print("Test completed successfully!")
print("="*80)

Output tokens shape: torch.Size([1, 40])
Output tokens:
tensor([[18075,     1,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0]],
       device='cuda:0')
Updated past key values: <transformers.cache_utils.DynamicCache object at 0x7e50cc953080>

High Level Prompt Task: Put the grapes on the basket.
Subtask:

Decoded output:
Batch 0: pick<eos>

Test completed successfully!


# Results

The model don't produce non sense words. But it just produce a word, maybe there is some bug with the EOS token. Or maybe the adaRMS thing. Before make a fine tunning we have to be sure about it.

### Fine-Tuning Example
To fine-tune the model, you need:
1. **Ground Truth Actions**: The actual movements the robot should make.
2. **Training Loop**: A standard PyTorch loop with an optimizer.

The following code demonstrates a single training step using dummy action data.

In [ ]:
import torch.optim as optim
from lerobot.utils.constants import ACTION

# 1. Setup Optimizer
# We fine-tune all parameters here, but you could also freeze specific layers
optimizer = optim.AdamW(policy.parameters(), lr=1e-5)

# 2. Prepare a Training Batch
# We start with the existing batch containing images and text
train_batch = batch.copy()

# We need to add Ground Truth Actions for the loss calculation
# Shape: (Batch_Size, Horizon, Action_Dim)
action_dim = policy.config.max_action_dim
horizon = policy.config.chunk_size
batch_size = 1

# Creating dummy actions (all zeros) for demonstration
# In reality, this would be your recorded robot data
dummy_actions = torch.zeros((batch_size, horizon, action_dim), device=device)
train_batch[ACTION] = dummy_actions

# Optional: Add mask for text loss (if you want to train the VLM part too)
# 1 = Calculate loss for this token, 0 = Ignore
# Typically you mask the prompt and only train on the generated response
train_batch["token_loss_mask"] = torch.ones_like(masks, device=device)

# 3. Run a Training Step
policy.train() # Switch to training mode
optimizer.zero_grad()

# The forward pass automatically calculates the loss
loss, loss_dict = policy(train_batch)

# Backward pass and optimization
loss.backward()
optimizer.step()

print("TRAINING STEP COMPLETE")
print(f"Total Loss: {loss.item():.4f}")
print(f"Detailed Losses: {loss_dict}")

In [ ]:
print("\n" + "="*80)
print("Testing sample_embedding...")
print("="*80)

with torch.no_grad():
    prefix_embeddings, past_key_values, prefix_pad_masks = policy.model.sample_embedding(
        processed_images,
        img_masks,
        tokens,
        masks
    )

    print(f"Prefix embeddings shape: {prefix_embeddings.shape}")
    print(f"Prefix pad masks shape: {prefix_pad_masks.shape}")
    print(f"Past key values type: {type(past_key_values)}")
    if past_key_values is not None:
        print(f"Past key values length: {len(past_key_values)}")
